In [1]:
pip install osmnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 2.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import osmnx as ox
import networkx as nx
import folium
import itertools
from geopy.distance import geodesic
import time
from typing import List, Dict, Tuple
import matplotlib.pyplot as plt
import requests
import json

# Configure settings
ox.settings.use_cache = True
ox.settings.log_console = True

In [ ]:
data = {
    "hospital": {
        "name": "Central Hospital",
        "latitude": 29.99512653425452,
        "longitude": 31.68462840171934,
        "type": "destination"
    },
    "patients": [
        {"id": "DT", "name": "Patient DT", "latitude": 30.000417586266437, "longitude": 31.73960813272627},
        {"id": "GR", "name": "Patient GR", "latitude": 30.011344405285193, "longitude": 31.747827362371993},
        {"id": "R2", "name": "Patient R2", "latitude": 30.030388325206854, "longitude": 31.669231198639675},
        {"id": "R3_2", "name": "Patient R3_2", "latitude": 30.030940768851426, "longitude": 31.688371339937028},
        {"id": "IT", "name": "Patient IT", "latitude": 30.01285635906825, "longitude": 31.693811715848444}
    ]
}

In [25]:
hospital = {"latitude": data["hospital"]["latitude"], "longitude": data["hospital"]["longitude"]}

patients = {p["id"]: {"latitude": p["latitude"], "longitude": p["longitude"]} for p in data["patients"]}


In [4]:
hospital_df = pd.DataFrame([data["hospital"]])
patients_df = pd.DataFrame(data["patients"])
patients_df["type"] = "patient"
locations_df = pd.concat([hospital_df, patients_df], ignore_index=True)
locations_df.drop_duplicates(subset=["latitude", "longitude"], inplace=True)
locations_df.reset_index(drop=True, inplace=True)

print("Locations DataFrame:")
locations_df

Locations DataFrame:


,name,latitude,longitude,type,id
0,Central Hospital,29.995127,31.684628,destination,NaN
1,Patient DT,30.000418,31.739608,patient,DT
2,Patient GR,30.011344,31.747827,patient,GR
3,Patient R2,30.030388,31.669231,patient,R2
4,Patient R3_2,30.030941,31.688371,patient,R3_2
5,Patient IT,30.012856,31.693812,patient,IT


In [ ]:
# function to get best route between two points using OSRM API
def best_route_between_points(lat1, lon1, lat2, lon2, return_geometry=False):
   
    coords_str = f"{lon1},{lat1};{lon2},{lat2}"
    url = f"http://router.project-osrm.org/route/v1/driving/{coords_str}?overview=full&geometries=geojson"

    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        data = response.json()

        if data['code'] == 'Ok' and data['routes']:
            route = data['routes'][0]
            distance_km = route['distance'] / 1000  # meters → km
            duration_min = route['duration'] / 60   # seconds → minutes
            geometry = route['geometry']['coordinates']

            # Debug print
            print(f"Best route from ({lat1}, {lon1}) → ({lat2}, {lon2})")
            print(f"  Distance: {distance_km:.2f} km")
            print(f"  Duration: {duration_min:.2f} min")

            if return_geometry:
                return distance_km, duration_min, geometry
            return distance_km, duration_min
        else:
            print("OSRM error: No route found")
            return None

    except requests.exceptions.RequestException as e:
        print(f"OSRM API error: {e}")
        return None

# function to visualize route on a folium map
def show_route_on_map(lat1, lon1, lat2, lon2):
    """
    Show best route between two points on a folium map.
    """
    result = best_route_between_points(lat1, lon1, lat2, lon2,return_geometry=True)
    if not result:
        return None
    _, _, geometry = result

    # Center map between points
    m = folium.Map(location=[(lat1+lat2)/2, (lon1+lon2)/2], zoom_start=13)
    folium.Marker([lat1, lon1], popup="Start", icon=folium.Icon(color="green")).add_to(m)
    folium.Marker([lat2, lon2], popup="End", icon=folium.Icon(color="red")).add_to(m)

    # Add route polyline
    folium.PolyLine([(lat, lon) for lon, lat in geometry], color="blue", weight=5, opacity=0.7).add_to(m)
    return m


In [6]:
# Coordinates: hospital and Patient DT
# Example: Hospital → Patient DT
lat_h, lon_h = 29.99512653425452, 31.68462840171934
lat_p, lon_p = 30.000417586266437, 31.73960813272627

# Just get distance and duration
dist, dur = best_route_between_points(lat_h, lon_h, lat_p, lon_p)
print("Returned distance:", dist, "km")
print("Returned duration:", dur, "min")

# Get distance, duration + geometry (for drawing route)
dist, dur, geom = best_route_between_points(lat_h, lon_h, lat_p, lon_p, return_geometry=True)
print("Geometry points:", len(geom))

Best route from (29.99512653425452, 31.68462840171934) → (30.000417586266437, 31.73960813272627)
  Distance: 8.63 km
  Duration: 13.18 min
Returned distance: 8.6285 km
Returned duration: 13.183333333333334 min
Best route from (29.99512653425452, 31.68462840171934) → (30.000417586266437, 31.73960813272627)
  Distance: 8.63 km
  Duration: 13.18 min
Geometry points: 179


In [ ]:
# Show map
print(f"Best route from Hopital → Patient DT")
map_pp = show_route_on_map(lat_h, lon_h, lat_p, lon_p)
map_pp.save("hospital_to_DT.html")
map_pp


Best route from Hopital → Patient DT
Best route from (29.99512653425452, 31.68462840171934) → (30.000417586266437, 31.73960813272627)
  Distance: 8.63 km
  Duration: 13.18 min


In [8]:
# Coordinates: hospital and Patient IT
# Example: Hospital → Patient IT
lat_h, lon_h = 29.99512653425452, 31.68462840171934
lat_p, lon_p = 30.01285635906825, 31.693811715848444
# Just get distance and duration
dist, dur = best_route_between_points(lat_h, lon_h, lat_p, lon_p)
print("Returned distance:", dist, "km")
print("Returned duration:", dur, "min")

# Get distance, duration + geometry (for drawing route)
dist, dur, geom = best_route_between_points(lat_h, lon_h, lat_p, lon_p, return_geometry=True)
print("Geometry points:", len(geom))

Best route from (29.99512653425452, 31.68462840171934) → (30.01285635906825, 31.693811715848444)
  Distance: 9.67 km
  Duration: 12.29 min
Returned distance: 9.6724 km
Returned duration: 12.295 min
Best route from (29.99512653425452, 31.68462840171934) → (30.01285635906825, 31.693811715848444)
  Distance: 9.67 km
  Duration: 12.29 min
Geometry points: 228


In [9]:

# Show map
print(f"Best route from Hopital → Patient IT")
map_pp = show_route_on_map(lat_h, lon_h, lat_p, lon_p)
map_pp.save("hospital_to_DT.html")
map_pp


Best route from Hopital → Patient IT
Best route from (29.99512653425452, 31.68462840171934) → (30.01285635906825, 31.693811715848444)
  Distance: 9.67 km
  Duration: 12.29 min


In [10]:
# Coordinates: hospital and Patient GR
# Example: Hospital → Patient GR

lat_h, lon_h = 29.99512653425452, 31.68462840171934
lat_p, lon_p = 30.011344405285193, 31.747827362371993
# Just get distance and duration
dist, dur = best_route_between_points(lat_h, lon_h, lat_p, lon_p)
print("Returned distance:", dist, "km")
print("Returned duration:", dur, "min")

# Get distance, duration + geometry (for drawing route)
dist, dur, geom = best_route_between_points(lat_h, lon_h, lat_p, lon_p, return_geometry=True)
print("Geometry points:", len(geom))

Best route from (29.99512653425452, 31.68462840171934) → (30.011344405285193, 31.747827362371993)
  Distance: 11.50 km
  Duration: 15.42 min
Returned distance: 11.4958 km
Returned duration: 15.42 min
Best route from (29.99512653425452, 31.68462840171934) → (30.011344405285193, 31.747827362371993)
  Distance: 11.50 km
  Duration: 15.42 min
Geometry points: 272


In [11]:
# Show map
print(f"Best route from Hopital → Patient GR")
map_pp = show_route_on_map(lat_h, lon_h, lat_p, lon_p)
map_pp.save("hospital_to_DT.html")

map_pp


Best route from Hopital → Patient GR
Best route from (29.99512653425452, 31.68462840171934) → (30.011344405285193, 31.747827362371993)
  Distance: 11.50 km
  Duration: 15.42 min


In [12]:
# Coordinates: hospital and Patient R2
# Example: Hospital → Patient R2

lat_h, lon_h = 29.99512653425452, 31.68462840171934
lat_p, lon_p = 30.030388325206854,31.669231198639675
# Just get distance and duration
dist, dur = best_route_between_points(lat_h, lon_h, lat_p, lon_p)
print("Returned distance:", dist, "km")
print("Returned duration:", dur, "min")

# Get distance, duration + geometry (for drawing route)
dist, dur, geom = best_route_between_points(lat_h, lon_h, lat_p, lon_p, return_geometry=True)
print("Geometry points:", len(geom))

Best route from (29.99512653425452, 31.68462840171934) → (30.030388325206854, 31.669231198639675)
  Distance: 9.45 km
  Duration: 15.81 min
Returned distance: 9.4454 km
Returned duration: 15.813333333333333 min
Best route from (29.99512653425452, 31.68462840171934) → (30.030388325206854, 31.669231198639675)
  Distance: 9.45 km
  Duration: 15.81 min
Geometry points: 198


In [13]:
# Show map
print(f"Best route from Hopital → Patient R2")
map_pp = show_route_on_map(lat_h, lon_h, lat_p, lon_p)
map_pp.save("hospital_to_DT.html")

map_pp


Best route from Hopital → Patient R2
Best route from (29.99512653425452, 31.68462840171934) → (30.030388325206854, 31.669231198639675)
  Distance: 9.45 km
  Duration: 15.81 min


In [14]:

# Coordinates: hospital and Patient R3_2
# Example: Hospital → Patient R3_2

lat_h, lon_h = 29.99512653425452, 31.68462840171934
lat_p, lon_p = 30.030940768851426,31.688371339937028
# Just get distance and duration
dist, dur = best_route_between_points(lat_h, lon_h, lat_p, lon_p)
print("Returned distance:", dist, "km")
print("Returned duration:", dur, "min")

# Get distance, duration + geometry (for drawing route)
dist, dur, geom = best_route_between_points(lat_h, lon_h, lat_p, lon_p, return_geometry=True)
print("Geometry points:", len(geom))

Best route from (29.99512653425452, 31.68462840171934) → (30.030940768851426, 31.688371339937028)
  Distance: 10.85 km
  Duration: 16.44 min
Returned distance: 10.8524 km
Returned duration: 16.441666666666666 min
Best route from (29.99512653425452, 31.68462840171934) → (30.030940768851426, 31.688371339937028)
  Distance: 10.85 km
  Duration: 16.44 min
Geometry points: 233


In [15]:
# Show map

print(f"Best route from Hopital → Patient R3_2")
map_pp = show_route_on_map(lat_h, lon_h, lat_p, lon_p)
map_pp.save("hospital_to_DT.html")

map_pp


Best route from Hopital → Patient R3_2
Best route from (29.99512653425452, 31.68462840171934) → (30.030940768851426, 31.688371339937028)
  Distance: 10.85 km
  Duration: 16.44 min
